In [1]:
#Imports

import numpy as np
import matplotlib.pyplot as plt
import hera_cal.abscal as abscal
import uvtools.dspec as dspec 
import hera_pspec
from hera_pspec import pspecbeam
from hera_pspec import pspecdata
import itertools
import scipy 
from scipy import signal
import pickle
import copy
from hera_cal.utils import polnum2str, polstr2num, jnum2str, jstr2num
from hera_cal.io import HERAData, HERACal
from hera_cal.io import DataContainer 
from hera_cal import apply_cal
from hera_cal import io
from hera_cal import smooth_cal
from hera_cal import vis_clean
from hera_cal import redcal
from pyuvdata import UVFlag
import glob
import tqdm
import os
import shutil
from hera_cal import frf
import imp
from hera_pspec import utils as uvp_utils
from hera_pspec import plot as pspec_plot
from hera_pspec import grouping

/users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_sim/visibilities/__init__.py:22: UserWarning: HealVis failed to import.
  warnings.warn("HealVis failed to import.")
/users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_sim/visibilities/__init__.py:27: UserWarning: PRISim failed to import.
  warnings.warn("PRISim failed to import.")
/users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_sim/visibilities/__init__.py:33: UserWarning: VisGPU failed to import.
  warnings.warn("VisGPU failed to import.")
/users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_sim/__init__.py:36: FutureWarning: 
In the next major release, all HERA-specific variables will be removed from the codebase. The following variables will need to be accessed through new class-like structures to be introduced in the next major release: 

noise.HERA_Tsky_mdl
noise.HERA_BEAM_POLY
sigchain.HERA_NRAO_BANDPASS
rfi.HERA_RFI_STATIONS

Additionally, the next maj

In [2]:
#Loading Data, Model, and Flags

flag_files = [f"H1C_Flags/{jd}.flags.h5" for jd in [2458098,2458099,2458101,2458102,2458103,2458104,2458105,2458106,
                                                    2458107,2458108,2458109,2458110,2458111,2458112,2458113,2458114,
                                                    2458115,2458116]]
model_files = sorted(glob.glob('/lustre/aoc/projects/hera/aewallwi/time_dep_flagging/2458098/*.true.uvh5'))
data_files = sorted(glob.glob('/lustre/aoc/projects/hera/aewallwi/time_dep_flagging/2458098/*.true.uvh5'))

In [8]:
#Calculate Number of Days and Chunks
nfiles = []
for day,flag_file in enumerate(flag_files):
    file_number=0
    for data_file in zip(data_files):
        file_number += 1
    nfiles.append(file_number)
print('Number of Days',len(nfiles))
print('Number of Chunks',file_number)

Number of Days 18
Number of Chunks 32


In [ ]:
#Average Before Filter (ABF)

if not os.path.exists(f'AverageBeforeFilter'):
        os.mkdir(f'AverageBeforeFilter')
for file_number in range(len(nfiles)):
    for day in range(len(flag_files)):
        cfile = f"Day_{day}/data_{day}_{file_number}_smoothcal.uvh5"
        hd = HERAData(cfile)
        data_t,flags_t,nsamples_t = hd.read()
        #Load First Day
        if day==0:
            data_avg=DataContainer({bl:data_t[bl]*nsamples_t[bl]*(~flags_t[bl]) for bl in data_t})
            flags_avg =DataContainer({bl:flags_t[bl] for bl in data_t})
            nsamples_avg = DataContainer({bl:nsamples_t[bl]*(~flags_t[bl]) for bl in data_t})
        #Average Following Days
        else:
            for bl in data_avg:
                data_avg[bl] = data_avg[bl] + data_t[bl]*nsamples_t[bl]*(~flags_t[bl])
                flags_avg[bl] = flags_avg[bl] & flags_t[bl]
                nsamples_avg[bl] = nsamples_avg[bl] + nsamples_t[bl]*(~flags_t[bl])           
    for bl in data_avg:   
        data_avg[bl] = data_avg[bl]/nsamples_avg[bl]
        data_avg[bl][~np.isfinite(data_avg[bl])] = 0.0
    hd.update(data=data_avg, flags=flags_avg, nsamples=nsamples_avg)
    hd.write_uvh5(f'AverageBeforeFilter/data_avg_{file_number}.uvh5', clobber=True)
    
#Fourier Filter
for file_number in range(len(nfiles)):
    vc = vis_clean.VisClean(f'AverageBeforeFilter/data_avg_{file_number}.uvh5')
    vc.read()
    vc.vis_clean(standoff=100, min_dly=600, mode='dpss_leastsq', skip_if_flag_within_edge_distance=1,
                 flag_model_rms_outliers=True, max_contiguous_edge_flags=1)
    vc.write_filtered_data(filled_outfilename=f'AverageBeforeFilter/data_avg_{file_number}_filtered.uvh5',
                           clobber=True)

In [ ]:
#Average After Filter (AAF)

if not os.path.exists(f'AverageAfterFilter'):
        os.mkdir(f'AverageAfterFilter')
for file_number in range(len(nfiles)):
    for day in range(len(flag_files)):
        ffile = f"Day_{day}/data_{day}_{file_number}_filtered.uvh5"
        hd = HERAData(ffile)
        data_f,flags_f,nsamples_f = hd.read()
        #Load First Day
        if day==0:
            data_avg=DataContainer({bl:data_f[bl]*nsamples_f[bl]*(~flags_f[bl]) for bl in data_f})
            flags_avg =DataContainer({bl:flags_f[bl] for bl in data_f})
            nsamples_avg = DataContainer({bl:nsamples_f[bl]*(~flags_f[bl]) for bl in data_f})
        #Average Following Days
        else:
            for bl in data_avg:
                data_avg[bl] = data_avg[bl] + data_f[bl]*nsamples_f[bl]*(~flags_f[bl])
                flags_avg[bl] = flags_avg[bl] & flags_f[bl]
                nsamples_avg[bl] = nsamples_avg[bl] + nsamples_f[bl]*(~flags_f[bl])
    for bl in data_avg:   
        data_avg[bl] = data_avg[bl]/nsamples_avg[bl]
        data_avg[bl][~np.isfinite(data_avg[bl])] = 0.0
    hd.update(data=data_avg, flags=flags_avg, nsamples=nsamples_avg)
    hd.write_uvh5(f'AverageAfterFilter/data_avg_{file_number}_filtered.uvh5', clobber=True)

In [ ]:
#Combining Chunks(Time Averaging) of ABF & AAF Days

cavg_before = sorted(glob.glob('AverageBeforeFilter/data_avg_*_filtered.uvh5'))
frf.time_avg_data_and_write(cavg_before,
                            output_data='AverageBeforeFilter/data_avg.Cavg.uvh5',
                            t_avg=300, rephase=True, clobber=True, wgt_by_nsample=False)

cavg_after = sorted(glob.glob('AverageAfterFilter/data_avg_*_filtered.uvh5'))
frf.time_avg_data_and_write(cavg_after,
                            output_data='AverageAfterFilter/data_avg.Cavg.uvh5',
                            t_avg=300, rephase=True, clobber=True,wgt_by_nsample=False)